In [22]:
from rdkit import Chem
from rdkit.Chem import Draw
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import Descriptors
from rdkit.Chem import AllChem
from rdkit import DataStructs
from rdkit.Chem import PandasTools


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as widgets

In [23]:
#Import henrys law and wang data
Kh=pd.read_csv('HLC.csv')[['species','SMILES','Kh_mean','Kh_first']]
Wang=pd.read_csv('Wang_data_key_parts_with_names.csv')[['species','SMILES','LogS','MW','MLOGP','InChI_Key']]



In [24]:
#make both datasets panda
dfKh = pd.DataFrame(Kh)
dfKh
dfWang = pd.DataFrame(Wang)
dfKh



,species,SMILES,Kh_mean,Kh_first
0,oxygen,O=O,11.267713,11.330604
1,ozone,O=[O]=O,9.508258,9.210340
2,hydrogen atom,[H][H],12.859999,12.859999
3,hydrogen,[H],11.768784,11.761387
4,deuterium,[2H][2H],11.748648,11.748648
...,...,...,...,...
4627,tetramethyl lead,C[Pb](C)(C)C,11.042922,11.042922
4628,ethyltrimethylplumbane,CC[Pb](C)(C)C,NaN,NaN
4629,diethyldimethylplumbane,CC[Pb](C)(C)CC,NaN,NaN
4630,triethylmethylplumbane,CC[Pb](C)(CC)CC,NaN,NaN


In [25]:
#Remove all nan values
dfKh.isnull()
dfKh=dfKh.dropna(how = 'any')
dfWang.isnull()
dfWang=dfWang.dropna()  
dfWang

,species,SMILES,LogS,MW,MLOGP,InChI_Key
0,5-[[3-[[(2-Methylpropyl)amino]methyl]phenyl]su...,CC(C)CNCc1ccccc1S(=O)(=O)c1ccc(S(N)(=O)=O)s1,-1.9266,388.58,1.370,DLKQHBOKULLWDQ
1,5-(4-Methylphenyl)sulfonylthiophene-2-sulfonamide,NS(=O)(=O)c1ccc(S(=O)(=O)c2ccc(CN3CCOCC3)cc2)s1,-1.0607,402.56,0.610,UPMXNNIRAGDFEH
2,Prometryn,CSc1nc(NC(C)C)nc(NC(C)C)n1,-4.1000,241.41,2.893,QCKHVNQHBOGZER
3,2-(Diethylamino)propyl 4-aminobenzoate,CCN(CC)C(C)COC(=O)c1ccc(N)cc1,-2.3840,250.38,2.386,NYQDCVLCJXRDSK
4,7-Chloropteridine,Clc1cnc2cncnc2n1,-0.8700,166.58,0.390,NLYNUTMZTCLNOO
...,...,...,...,...,...,...
3659,"2-Nitro-5-phenylcyclohexane-1,3-dione",O=C1CC(c2ccccc2)CC(=O)C1[N+](=O)[O-],-1.6690,233.24,1.335,UMNZZEWMRNSNIH
3660,"2,4-Dihydroxy-5-pyrimidinylsulfamic acid",O=c1[nH]cc(NS(=O)(=O)O)c(=O)[nH]1,-1.7660,207.19,-1.657,ACVNMTOONYGQIK
3661,4-Chloro-2-(furan-2-ylmethylamino)-5-sulfamoyl...,NS(=O)(=O)c1cc(C(=O)O)c(NCc2ccco2)cc1Cl,-3.6600,330.77,0.434,LPTITAGPBXDDGR
3662,4-Chloro-2-(furan-2-ylmethylamino)-5-sulfamoyl...,NS(=O)(=O)c1cc(C(=O)O)c(NCc2ccco2)cc1Cl,-3.8250,330.77,0.434,RTHCYVBBDHJXIQ


In [26]:
#Set both indexs to SMILES so i can extract lists
dfKh=dfKh.set_index(['SMILES'])
dfKh
dfWang=dfWang.rename(columns={'smi':'SMILES'},)
dfWang=dfWang.set_index(['SMILES'])
dfWang


#Extract Smiles as a list
#Wang_smiles=dfKh.index.tolist()
#I thought if i change this line below to the one above id get the Inchikey for henry but it gets annoyed instead
Wang_smiles=dfWang.index.tolist()
Wang_smiles=np.transpose(Wang_smiles)
Wang_smiles = list(Wang_smiles)
Wang_smiles 


['CC(C)CNCc1ccccc1S(=O)(=O)c1ccc(S(N)(=O)=O)s1',
 'NS(=O)(=O)c1ccc(S(=O)(=O)c2ccc(CN3CCOCC3)cc2)s1',
 'CSc1nc(NC(C)C)nc(NC(C)C)n1',
 'CCN(CC)C(C)COC(=O)c1ccc(N)cc1',
 'Clc1cnc2cncnc2n1',
 'CN1C(=O)CN=C(c2ccccc2)c2cc(Cl)ccc21',
 'CN1C(=O)CN=C(c2ccccc2)c2cc(Cl)ccc21',
 'CN1C(=O)CN=C(c2ccccc2)c2cc(Cl)ccc21',
 'CC(=O)C1CCC2C3CCC4=CC(=O)CC(O)C4(C)C3CCC12C',
 'CC12CCC(=O)C=C1CCC1C3CCC(O)(C(=O)CO)C3(C)CC(O)C12F',
 'CC12CCC(=O)C=C1CCC1C3CCC(O)(C(=O)CO)C3(C)CC(O)C12F',
 'COc1ccccc1OC',
 'O=C1NC(=O)C([N+](=O)[O-])C(=O)N1',
 'CCCCCC(C)(C)O',
 'CCCCCC(C)(C)O',
 'O=c1[nH]c(CN2CCCC2)c(Br)c(=O)[nH]1',
 'O=C1Nc2ccc(Cl)cc2C(c2ccccc2)=NC1O',
 'O=C1CC2OCC=C3C[N+]4([O-])CCC56c7ccccc7N1C5C2C3CC64',
 'CC(=O)Nc1ccccc1O',
 'CC(O)C(=O)NCCCCCCNC(=O)C(C)O',
 'CC12CCC3C(CCC4=CC(=O)CCC43C)C1CCC2=O',
 'CCC(C)(C)CC',
 'CCCC(C)C',
 'CCCC(C)C',
 'CNc1ccccc1',
 'C=CCCCCCCCC',
 'CCCCNC(=O)NS(=O)(=O)c1ccccc1',
 'O=C(O)c1cccc([N+](=O)[O-])c1',
 'O=C(O)c1cccc([N+](=O)[O-])c1',
 'CC(C)(C)COC(N)=O',
 'COc1ccc(OC)c(C=O)c1',
 

In [27]:
#for loop to compile a list of molcules data from the SMILES
mol_list=[]
for Smiles in Wang_smiles:
    
    mol = Chem.MolFromSmiles(Smiles)
    mol_list.append(mol)
    
print(mol_list)

#img = Draw.MolsToGridImage(mol_list)
#img




[<rdkit.Chem.rdchem.Mol object at 0x000002406BC07350>, <rdkit.Chem.rdchem.Mol object at 0x000002406BC07760>, <rdkit.Chem.rdchem.Mol object at 0x000002406BC07850>, <rdkit.Chem.rdchem.Mol object at 0x000002406BC078A0>, <rdkit.Chem.rdchem.Mol object at 0x000002406BC078F0>, <rdkit.Chem.rdchem.Mol object at 0x000002406BC07940>, <rdkit.Chem.rdchem.Mol object at 0x000002406BC079E0>, <rdkit.Chem.rdchem.Mol object at 0x000002406BC07990>, <rdkit.Chem.rdchem.Mol object at 0x000002406BC07A80>, <rdkit.Chem.rdchem.Mol object at 0x000002406BC07A30>, <rdkit.Chem.rdchem.Mol object at 0x000002406BC07AD0>, <rdkit.Chem.rdchem.Mol object at 0x000002406BC07B20>, <rdkit.Chem.rdchem.Mol object at 0x000002406BC07B70>, <rdkit.Chem.rdchem.Mol object at 0x000002406BC07C10>, <rdkit.Chem.rdchem.Mol object at 0x000002406BC07C60>, <rdkit.Chem.rdchem.Mol object at 0x000002406BC07CB0>, <rdkit.Chem.rdchem.Mol object at 0x000002406BC07D00>, <rdkit.Chem.rdchem.Mol object at 0x000002406BC07D50>, <rdkit.Chem.rdchem.Mol obje

In [28]:
#Compile Lnchikeys from RDKit molecules

Inchi_list=[]
for Inchi in mol_list:
    
    if Inchi== None:
        Inchi_list.append('NaN')
    else:
        Chi = Chem.MolToInchiKey(Inchi)
        Inchi_list.append(Chi)
    
print(Inchi_list)

['AABOBTKVYJYOAX-UHFFFAOYSA-N', 'AACPZXPPANDZOJ-UHFFFAOYSA-N', 'AAEVYOVXGOFMJO-UHFFFAOYSA-N', 'AAKAAFFHKNBIKB-UHFFFAOYSA-N', 'AANUQVPVSJJCRI-UHFFFAOYSA-N', 'AAOVKJBEBIDNHE-UHFFFAOYSA-N', 'AAOVKJBEBIDNHE-UHFFFAOYSA-N', 'AAOVKJBEBIDNHE-UHFFFAOYSA-N', 'AARNXMRADQFOEB-UHFFFAOYSA-N', 'AAXVEMMRQDVLJB-UHFFFAOYSA-N', 'AAXVEMMRQDVLJB-UHFFFAOYSA-N', 'ABDKAPXRBAPSQN-UHFFFAOYSA-N', 'ABICJYZKIYUWEE-UHFFFAOYSA-N', 'ACBMYYVZWKYLIP-UHFFFAOYSA-N', 'ACBMYYVZWKYLIP-UHFFFAOYSA-N', 'ACVNMTOONYGQIK-UHFFFAOYSA-N', 'ADIMAYPTOBDMTL-UHFFFAOYSA-N', 'ADTDBAKUQAKBGZ-UHFFFAOYSA-N', 'ADVGKWPZRIDURE-UHFFFAOYSA-N', 'ADXAXAXRYZPBNQ-UHFFFAOYSA-N', 'AEMFNILZOJDQLW-UHFFFAOYSA-N', 'AEXMKKGTQYQZCS-UHFFFAOYSA-N', 'AFABGHUZZDYHJO-UHFFFAOYSA-N', 'AFABGHUZZDYHJO-UHFFFAOYSA-N', 'AFBPFSWMIHJQDM-UHFFFAOYSA-N', 'AFFLGGQVNFXPEV-UHFFFAOYSA-N', 'AFOGBLYPWJJVAL-UHFFFAOYSA-N', 'AFPHTEQTJZKQAQ-UHFFFAOYSA-N', 'AFPHTEQTJZKQAQ-UHFFFAOYSA-N', 'AFQYFVWMIRMBAE-UHFFFAOYSA-N', 'AFUKNJHPZAVHGQ-UHFFFAOYSA-N', 'AFVRYMPIZFJJIL-UHFFFAOYSA-N', 'AFYCEA

In [29]:
#add missing LNchiKeys to dfKn data set
dfKh['InChI_Key']=Inchi_list
dfKh


#dfWang.to_csv('WangSmiles_Inchi.csv')


ValueError: Length of values (3663) does not match length of index (2297)

In [30]:
#make Lnchikeys the index so the a common value to merge tables
dfWang=dfWang.set_index(['InChI_Key'])
dfKh=dfKh.set_index(['InChI_Key'])

KeyError: "None of ['InChI_Key'] are in the columns"

In [34]:
result = pd.concat([dfWang, dfKh], axis=0, sort=False)
result

,species,LogS,MW,MLOGP,Kh_mean
InChI_Key,,,,,
AABOBTKVYJYOAX-UHFFFAOYSA-N,5-[[3-[[(2-Methylpropyl)amino]methyl]phenyl]su...,-1.9266,388.58,1.370,NaN
AACPZXPPANDZOJ-UHFFFAOYSA-N,5-(4-Methylphenyl)sulfonylthiophene-2-sulfonamide,-1.0607,402.56,0.610,NaN
AAEVYOVXGOFMJO-UHFFFAOYSA-N,Prometryn,-4.1000,241.41,2.893,NaN
AAKAAFFHKNBIKB-UHFFFAOYSA-N,2-(Diethylamino)propyl 4-aminobenzoate,-2.3840,250.38,2.386,NaN
AANUQVPVSJJCRI-UHFFFAOYSA-N,7-Chloropteridine,-0.8700,166.58,0.390,NaN
...,...,...,...,...,...
AWGTVRDHKJQFAX-UHFFFAOYSA-M,chlorophenylmercury,NaN,NaN,NaN,-5.940171
ZDHHIJSLJCLMPX-UHFFFAOYSA-M,bromomethylmercury,NaN,NaN,NaN,-1.308333
JVDIOYBHEYUIBM-UHFFFAOYSA-M,iodomethylmercury,NaN,NaN,NaN,0.544727


In [35]:
#return final table
result.isnull()
result=result.dropna()
result

,species,LogS,MW,MLOGP,Kh_mean
InChI_Key,,,,,


In [10]:
mol=Chem.MolFromSmiles('O=[O]=O')
mol